## LLaMA 2 指令微调（Alpaca-Style on Dolly-15K Dataset)

示例代码关键训练要素：
- 使用 Dolly-15K 数据集，以 Alpaca 指令风格生成训练数据
- 以 4-bit（NF4）量化精度加载 `LLaMA 2-7B` 模型
- 使用 QLoRA 以 `bf16` 混合精度训练模型
- 使用 `HuggingFace TRL` 的 `SFTTrainer` 实现监督指令微调
- 使用 Flash Attention 快速注意力机制加速训练（需硬件支持）

### 下载 databricks-dolly-15k 数据集

In [1]:
from datasets import load_dataset
from random import randrange
 
# 从hub加载数据集
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 15011 examples [00:00, 237467.46 examples/s]


In [2]:
# 数据集样例总数: 15011
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [3]:
# 随机抽选一个数据样例打印
print(dataset[randrange(len(dataset))])

{'instruction': "Given this paragraph about Grumpy Cat, tell me the cat's real name.", 'context': 'Tardar Sauce (April 4, 2012 – May 14, 2019), nicknamed Grumpy Cat, was an American Internet celebrity cat. She was known for her permanently "grumpy" facial appearance, which was caused by an underbite and feline dwarfism. She came to prominence when a photograph of her was posted on September 22, 2012, on social news website Reddit by Bryan Bundesen, the brother of her owner Tabatha Bundesen. "Lolcats" and parodies created from the photograph by Reddit users became popular. She was the subject of a popular Internet meme in which humorously negative, cynical images were made from photographs of her.', 'response': 'Grumpy Cat was actually a nickname for the famous internet celebrity meme cat. Her real name was Tardar Sauce.', 'category': 'closed_qa'}


### 以 Alpaca-Style 格式化指令数据

`Alpacca-style` 格式：https://github.com/tatsu-lab/stanford_alpaca#data-release

In [4]:
def format_instruction(sample_data):
    """
    Formats the given data into a structured instruction format.

    Parameters:
    sample_data (dict): A dictionary containing 'response' and 'instruction' keys.

    Returns:
    str: A formatted string containing the instruction, input, and response.
    """
    # Check if required keys exist in the sample_data
    if 'response' not in sample_data or 'instruction' not in sample_data:
        # Handle the error or return a default message
        return "Error: 'response' or 'instruction' key missing in the input data."

    return f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 
 
### Input:
{sample_data['response']}
 
### Response:
{sample_data['instruction']}
"""

In [5]:
# 随机抽选一个样例，打印 Alpaca 格式化后的样例 
print(format_instruction(dataset[randrange(len(dataset))]))

### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 
 
### Input:
Great New England Patriots football players include: Tom Brady, Rob Gronkowski, Ron Burton, Ty Law, Teddy Bruschi, and John Hannah.
 
### Response:
What are the names of some great New England Patriots football players?



### 使用快速注意力（Flash Attention）加速训练

检查你的 GPU 是否支持 `flash-attn` 加速：

```shell
$ python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
AssertionError: Hardware not supported for Flash Attention
```
**运行结果：演示使用的 NVIDIA T4 硬件不支持 Flash Attention**

#### 安装 flash-attn 加速包（需要GPU硬件支持）

```shell
$ MAX_JOBS=4 pip install flash-attn --no-build-isolation
```

### 加载模型

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 如果硬件设备支持，成功安装 flash-attn后，将 use_flash_attention 设置为True
use_flash_attention = False
 
# 取消注释以使用 flash-atten
# if torch.cuda.get_device_capability()[0] >= 8:
#     from utils.llama_patch import replace_attn_with_flash_attn
#     print("Using flash attention")
#     replace_attn_with_flash_attn()
#     use_flash_attention = True
 
 
# 获取 LLaMA 2-7B 模型权重
# 无需 Meta AI 审核的模型权重
model_id = "NousResearch/Llama-2-7b-hf" 
# 通过 Meta AI 审核后可使用此 Model ID 下载
# model_id = "meta-llama/Llama-2-7b-hf" 
 
 
# 使用 BnB 加载量化后的模型
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
 
# 加载模型与分词器
# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="cpu", torch_dtype="auto")
# model.config.pretraining_tp = 1 
 
# # 通过对比doc中的字符串，验证模型是否在使用flash attention
# if use_flash_attention:
#     from utils.llama_patch import forward    
#     assert model.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"


# 1. 删掉 bnb_config，不再量化
# 2. 强制 device_map="cpu" 或 "auto"（无 CUDA 时会自动回落 CPU）
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",      # 自动选 fp32/fp16
    device_map="cpu",        # 显式 CPU
    use_cache=False
)
 
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

  2025-10-15T06:00:39.475811Z  WARN  Reqwest(reqwest::Error { kind: Request, url: "https://transfer.xethub.hf.co/xorbs/default/5ac7292045c700f3080b4d6877a4772b8d093338fd8fef71683d60681cb24454?X-Xet-Signed-Range=bytes%3D45898883-67064864&X-Xet-Session-Id=01K7K6TFP2PFFD016K7CEGEK2K&Expires=1760511430&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly90cmFuc2Zlci54ZXRodWIuaGYuY28veG9yYnMvZGVmYXVsdC81YWM3MjkyMDQ1YzcwMGYzMDgwYjRkNjg3N2E0NzcyYjhkMDkzMzM4ZmQ4ZmVmNzE2ODNkNjA2ODFjYjI0NDU0P1gtWGV0LVNpZ25lZC1SYW5nZT1ieXRlcyUzRDQ1ODk4ODgzLTY3MDY0ODY0JlgtWGV0LVNlc3Npb24tSWQ9MDFLN0s2VEZQMlBGRkQwMTZLN0NFR0VLMksiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjA1MTE0MzB9fX1dfQ__&Signature=ADAWDBQy7qP4uKFwLXtlsEmy7Br7dU7aCO0n9vbvJD6-YvBBOTjixYZDah010mKm9Pc3IIWnNnmrlcxjlyWHA0IZRPSCd2nJAu70nu2GWq8nFu10dN2lHaom424D0RFqXhX9Yo~ucqb73yfSTfq0f92YlBtZnPQgKVwUonUghcAVdkNiDpdWJ85qWyKFL4jXsA43Br0mk2Ek9ZPr7s7IdBky74K7T8U~Po30izqDt3DSk1vbX3djdOtuTVoElQrrPkF6Q7iRleobzcrIlJVxwcWLU639QMQGy1pR1VMALI

  2025-10-15T06:29:58.622935Z  WARN  Reqwest(reqwest::Error { kind: Request, url: "https://transfer.xethub.hf.co/xorbs/default/14631ffd628f89debaa73584719cccd4f2f71df93540aae3dac03ecc627addf8?X-Xet-Signed-Range=bytes%3D64150041-65993540&X-Xet-Session-Id=01K7K6TFP2PFFD016K7CEGEK2K&Expires=1760511431&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly90cmFuc2Zlci54ZXRodWIuaGYuY28veG9yYnMvZGVmYXVsdC8xNDYzMWZmZDYyOGY4OWRlYmFhNzM1ODQ3MTljY2NkNGYyZjcxZGY5MzU0MGFhZTNkYWMwM2VjYzYyN2FkZGY4P1gtWGV0LVNpZ25lZC1SYW5nZT1ieXRlcyUzRDY0MTUwMDQxLTY1OTkzNTQwJlgtWGV0LVNlc3Npb24tSWQ9MDFLN0s2VEZQMlBGRkQwMTZLN0NFR0VLMksiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjA1MTE0MzF9fX1dfQ__&Signature=CS-IBOe4glYnwGxv4VZnmMM9Zo2YEZY2pJeh4yInvQ~R~yBde8g0o2CXpptEAn75OIX0DSsFyeUCn9QJt5Vmy0gTF6lliJKiswecFXZqe7iI3VFThM5EioHRDR4W3mofZR-db27MRGJkCeEExy9elIq~iDwnoHBeita7~Wv1iZxeMzZDKt6lO8Cr5GO5pXC-sTXCp-zsXeQ97Adey8Ah1nNMtlzifBDy2WBPjeQXeVnTmRBXreeNY~P7hru~5snGcQZIv05b9OkkR5YmqUeAi83HY3IvMTW8-e91tSge1q

  2025-10-15T06:29:59.149431Z  WARN  Reqwest(reqwest::Error { kind: Request, url: "https://transfer.xethub.hf.co/xorbs/default/7c3f347482e1a875e8187824b2cb9243ade32d77cfc0a5e4f2de57d7ad37f0d8?X-Xet-Signed-Range=bytes%3D24593448-58076177&X-Xet-Session-Id=01K7K6TFP2PFFD016K7CEGEK2K&Expires=1760511149&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly90cmFuc2Zlci54ZXRodWIuaGYuY28veG9yYnMvZGVmYXVsdC83YzNmMzQ3NDgyZTFhODc1ZTgxODc4MjRiMmNiOTI0M2FkZTMyZDc3Y2ZjMGE1ZTRmMmRlNTdkN2FkMzdmMGQ4P1gtWGV0LVNpZ25lZC1SYW5nZT1ieXRlcyUzRDI0NTkzNDQ4LTU4MDc2MTc3JlgtWGV0LVNlc3Npb24tSWQ9MDFLN0s2VEZQMlBGRkQwMTZLN0NFR0VLMksiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjA1MTExNDl9fX1dfQ__&Signature=wF9Ym4Mr5PGVYGVqPPjT5PwdO0HUYzrpiObVXBNuJEzU0NTDOOdKZMuYR7qoSxTKbPKu9hov421zSJqvg8KmvpTOJlsPfW7A883PtVA3Srp2UVu4BXTjPJ3Ja42YCO3I1sGlxgQineRPOXL~MT94wtqdhXVu1Lvm~6TZLvAqPcDLMdM1xduDOlZbIXe8FYlKEJciEr5EF9xpH8djHcE~rK0qxkj-D4H8P0b2JwuKZdnbXygUHG~xvrMoelqvS9ogeypv5Ux8kLATCXN5R6QGUVH9m3jeXFz5X-dcVZkvQN

RuntimeError: Data processing error: CAS service error : ReqwestMiddleware Error: Request failed after 5 retries

  2025-10-15T06:30:00.932433Z  WARN  Reqwest(reqwest::Error { kind: Request, url: "https://transfer.xethub.hf.co/xorbs/default/cc2eb3581379e22e590aee37efe994df1f52fce0a3630bf1d91b9de4d7b14dab?X-Xet-Signed-Range=bytes%3D43057022-66988652&X-Xet-Session-Id=01K7K6TFP2PFFD016K7CEGEK2K&Expires=1760511431&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly90cmFuc2Zlci54ZXRodWIuaGYuY28veG9yYnMvZGVmYXVsdC9jYzJlYjM1ODEzNzllMjJlNTkwYWVlMzdlZmU5OTRkZjFmNTJmY2UwYTM2MzBiZjFkOTFiOWRlNGQ3YjE0ZGFiP1gtWGV0LVNpZ25lZC1SYW5nZT1ieXRlcyUzRDQzMDU3MDIyLTY2OTg4NjUyJlgtWGV0LVNlc3Npb24tSWQ9MDFLN0s2VEZQMlBGRkQwMTZLN0NFR0VLMksiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NjA1MTE0MzF9fX1dfQ__&Signature=djP0aT1xludjNK~LMXwcCEfP71aITc7wFiK1OBpnKRY9eONeBXWU1rug4J99tPQNsgUBP3xGFXd5ckkptvUlPMMdOY55g3MZU07D3JUjpD0TEkv3q7PCChe9zODvpRDj6nGR97kL61koSOUuDSUdknV6eNbIu32Z36NzJTXMPg757Rm4EEYKGc08WeK3YoNErbapjX7myu~hkFsoD10DzkFpaEan~6iaPjzmb~TmvH2BZyppU20JOxzVJpCNbW4m8xAzZOt~cyTt8CAGAjXqdyTlgSWy6pBpp~xIAFqJaA

not have GPU ,so not do success

### 使用 QLoRA 配置加载 PEFT 模型

In [7]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
 
# QLoRA 配置
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=16,
        bias="none",
        task_type="CAUSAL_LM", 
)
 
 
# 使用 QLoRA 配置加载 PEFT 模型
model = prepare_model_for_kbit_training(model)
qlora_model = get_peft_model(model, peft_config)

In [8]:
qlora_model.print_trainable_parameters()

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.12433454005023165


### 训练超参数

In [9]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# 演示训练参数（实际训练是设置为 False）
demo_train = True
output_dir = f"models/llama-7-int4-dolly-{timestamp}"

In [10]:
from transformers import TrainingArguments
 
args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1 if demo_train else 3,
    max_steps=100,
    per_device_train_batch_size=3, # Nvidia T4 16GB 显存支持的最大 Batch Size
    gradient_accumulation_steps=1 if demo_train else 4,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="steps" if demo_train else "epoch",
    save_steps=10,
    learning_rate=2e-4,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant"
)

### 实例化 SFTTrainer

In [11]:
from trl import SFTTrainer
 
# 数据集的最大长度序列（筛选后的训练数据样例数为1158）
max_seq_length = 2048 
 
trainer = SFTTrainer(
    model=qlora_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction, 
    args=args,
)

[2024-03-09 00:05:43,454] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/root/miniconda3/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### 训练模型

In [12]:
trainer.train()

/root/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.561400
20,1.421200
30,1.338200
40,1.276900
50,1.252400
60,1.221700
70,1.274100
80,1.223100
90,1.244000
100,1.180300


Checkpoint destination directory models/llama-7-int4-dolly/checkpoint-10 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/root/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory models/llama-7-int4-dolly/checkpoint-20 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/root/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if u

TrainOutput(global_step=100, training_loss=1.299329252243042, metrics={'train_runtime': 11487.4223, 'train_samples_per_second': 0.026, 'train_steps_per_second': 0.009, 'total_flos': 2.43882352705536e+16, 'train_loss': 1.299329252243042, 'epoch': 0.26})

### 保存模型

In [14]:
trainer.save_model()

### 模型推理（测试）

In [ ]:
qlora_model